# Amazon RDS Proxy
- AWS에서 제공하는 RDS의 Connection Pool 서비스
    - Connection Pool : 커넥션 풀은 데이터베이스와의 연결을 관리하고 재사용하여 어플리케이션의 성능을 향상시키는 기능
    - RDS로 연결하는 Connection들을 중간에서 관리하여 효율적으로 배분 및 관리
- AWS의 EC2 혹은 Lambda와 같이 대규모의 Connection이 필요한 경우 활용 가능
- 유저/패스워드 인증 혹은 IAM 인증 지원
- 주의 사항
    - VPC 내부에서만 활용 가능 : 퍼블릭 인터넷에서 접근 불가능(데이터베이스의 접근 가능 여부와 무관)
    - 별도의 엔드포인트 보유 : RDS와 다른 엔드포인트 사용 필요
        - 추가로 엔드포인트 생성 가능
    - 기타 DB 엔진 별로 포트 제한이나 제약 사항이 있을 수 있음.

## RDS Proxy와 Secrets Manager
- RDS Proxy는 Secrets Manager를 활용해서 RDS에 접근
    - 즉 Secrets Manager에 RDS 자격증명 보관 필요
    - 각 유저별로 다른 Secret 생성 필요
- IAM DB 인증과 무관
    - 즉 RDS Proxy는 IAM DB 인증을 사용하더라도, RDS Proxy에서 RDS는 Secrets Manager 활용
    - RDS Proxy에 IAM DB 인증을 통해 접근할 뿐, 실제 Userid/Password(Secrets Manager)를 통해 RDS에 접근


### 데모 - RDS Proxy 사용
- RDS provision
    - Secrets Manager 활용
- RDS Proxy 설정
- node.js 기반 어플리케이션으로 접속 테스트
    - Secrets Manager를 활용해 인증 정보를 받아와 RDS와 통신
    - RDS Proxy를 사용해 RDS와 통신
    - RDS Proxy + IAM 인증을 사용해 RDS와 통신

### 실습 순서
1. RDS 프로비전
    - MySQL
    - 프리티어
    - 데이터베이스 이름 my-db
    - 자격 증명 관리 Secrets Manager 선택
    - 퍼블릭 엑세스 yes
2. IAM 권한 생성
    - 역할 생성
        - EC2가 사용
        - 권한 : secretsmanagerreadwrite
        - 권한 추가 : 인라인 정책, json에 스크립트 넣기
3. Secrets Manager
    - 보안 암호 세부 정보
    - 보안 암호값 검색 : 나중에 이거 활용할 거임
4. EC2 인스턴스 프로비전
    - 키페어 사용
    - 기존 보안 그룹 디폴트 선택
    - 고급 세부 정보 : IAM 인스턴스 프로필 적용
    - 유저데이터 입력 : 실습 스크립트
5. RDS Proxy 프로비전
    - 프록시 식별자 : my-db-proxy
    - 대상 그룹 : my-db
    - 인증 : secrets manager 보안 암호 선택
    - IAM 인증 허용되지 않음
    - 전송 계층 보안 선택
6. .env 파일 수정, 인스턴스에 파일 넘기기
7. 인스턴스에서 어플리케이션 실행, 각종 테스트
8. rds proxy를 수정, iam db 인증으로 접근해보자
    - iam 인증 필수로 변경
9. 리소스 정리 : 인스턴스, 프록시, 데이터베이스, secrets manager